In [1]:
%matplotlib

Using matplotlib backend: Qt5Agg


In [2]:
%matplotlib inline

In [1]:
pd.set_option('display.max_rows',    100)      # DataFrame 데이터 확인 시 최대 표시 행 수
pd.set_option('display.max_columns', 100)      # DataFrame 데이터 확인 시 최대 표시 열 수

NameError: name 'pd' is not defined

## 라이브러리 import

In [2]:
import pandas as pd
import numpy as np

In [3]:
import tensorflow as tf

tf.__version__                         # tensorflow 버전 확인

'2.0.0'

Tensorflow v1 기능만 사용 (v2 기능 강제 차단)

In [4]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.set_random_seed(777)

Instructions for updating:
non-resource variables are not supported in the long term


## 데이터 불러오기

In [5]:
Fold = 4

# k-fold 학습/검증 데이터
for i in range(Fold):
    
    path1 = './K_FoldData/Training_Fold%d'%(i+1)
    path2 = './K_FoldData/Validation_Fold%d'%(i+1)
    c1 = 'Training_Fold%d   = np.array(pd.read_csv(path1, sep=",", header=None))'%(i+1)
    c2 = 'Validation_Fold%d = np.array(pd.read_csv(path2, sep=",", header=None))'%(i+1)
    exec(c1)
    exec(c2)

# K-fold 학습/검증 레이블
TrainingFold_Label   = np.array(pd.read_csv('./K_FoldData/TrainingFold_Label_forANN'  , sep=",", header=None))
ValidationFold_Label = np.array(pd.read_csv('./K_FoldData/ValidationFold_Label_forANN', sep=",", header=None))
    
    
# 전체 학습용 데이터
Training_All       = np.array(pd.read_csv('./K_FoldData/Training_All', sep = ",", header = None))
Training_All_Label = np.array(pd.read_csv('./K_FoldData/Training_All_Label_forANN', sep = ",", header = None))

print(Training_Fold1.shape)
print(Validation_Fold1.shape)
print(TrainingFold_Label.shape)
print(ValidationFold_Label.shape)
print(Training_All.shape)

(256, 5)
(64, 5)
(256, 2)
(64, 2)
(320, 5)


## ANN(Artificial Neural Network) hyperparameter 설정

In [6]:
learningRate  = 0.0001
noOfNeuron    = 16
iteration     = 5000

## ANN 구조(Architecture) 설계

In [7]:
# input place holders
X = tf.placeholder(tf.float32, [None, Training_Fold1.shape[1]])
Y = tf.placeholder(tf.float32, [None, 2])

# dropout (keep_prob) rate 0.7 on training, but should be 1 for testing
keep_prob = tf.placeholder(tf.float32)

# weights & bias for nn layers

# Input Layer w: weight b: bias l: layer
W1 = tf.Variable(tf.random_normal([Training_Fold1.shape[1], noOfNeuron]))
b1 = tf.Variable(tf.random_normal([noOfNeuron]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

# Hidden Layer 1
W2 = tf.Variable(tf.random_normal([noOfNeuron, noOfNeuron]))
b2 = tf.Variable(tf.random_normal([noOfNeuron]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

# Hidden Layer 2
W3 = tf.Variable(tf.random_normal([noOfNeuron, noOfNeuron]))
b3 = tf.Variable(tf.random_normal([noOfNeuron]))
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)

# Output Layer
W4 = tf.Variable(tf.random_normal([noOfNeuron, 2]))
b4 = tf.Variable(tf.random_normal([2]))
hypothesis = tf.matmul(L3, W4) + b4


# define cost/loss &optimizer  cost함수, optimizer: 원래는 gradient descent 썼는데 AdamOptimizer가 대세
cost      = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learningRate).minimize(cost)

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy           = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## NN(Neural Network) 학습 및 평가 방법

In [8]:
Data      = Training_Fold1
Data_Val  = Validation_Fold1
Label     = TrainingFold_Label
Label_Val = ValidationFold_Label

# session 텐서플로의 그래프 지정
sess = tf.Session()
sess.run(tf.global_variables_initializer()) # 변수들 초기화

for step in range(1, iteration+1): #반복횟수 설정해서 계속 훈련
    # x, y는 input data임
    c, _ = sess.run([cost, optimizer], feed_dict = {X : Data, Y: Label}) # _ : optimizer. 필요 없어서 이렇게 넣음
    

Validation_Acc_Fold1= sess.run(accuracy, feed_dict={X : Data_Val, Y: Label_Val})
Validation_Acc_Fold1

0.515625

## NN(Neural Network) 학습 및 검증

In [9]:
Label     = TrainingFold_Label
Label_Val = ValidationFold_Label
Accuracy_sum = 0

print('[Result of K-fold Cross Validation] \n')

for i in range(Fold):
    
    s1= 'Data     = Training_Fold%d'  %(i+1)
    s2= 'Data_Val = Validation_Fold%d'%(i+1)
    exec(s1)
    exec(s2)

# train my model
# initialize
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    for step in range(1, iteration+1):
    
        c, _ = sess.run([cost, optimizer], feed_dict = {X : Data, Y: Label})
        
    Accuracy = sess.run(accuracy, feed_dict={X : Data_Val, Y: Label_Val})
    Accuracy_sum = Accuracy_sum + Accuracy
    
    print('Fold {}: {:.2f}%'.format((i+1), Accuracy*100))
    
print('* Average accuracy : {:.2f}%'.format((Accuracy_sum/Fold)*100))

[Result of K-fold Cross Validation] 

Fold 1: 51.56%
Fold 2: 56.25%
Fold 3: 65.62%
Fold 4: 53.12%
* Average accuracy : 56.64%


## 전체 학습 데이터로 NN 학습 및 모델 저장

In [10]:
x_data = Training_All
y_data = Training_All_Label

In [11]:
# train my model
# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step in range(1, iteration+1):
    
    c, o = sess.run([cost, optimizer], feed_dict = {X : x_data, Y: y_data})
    Acc = sess.run(accuracy, feed_dict={X : x_data, Y: y_data})
   
    if step % 1000 == 0:
            
        print('iteration {}'.format(step))    
        print('   Validation Acc: {:.2f}%'.format(Acc*100))

iteration 1000
   Validation Acc: 49.38%
iteration 2000
   Validation Acc: 53.44%
iteration 3000
   Validation Acc: 48.75%
iteration 4000
   Validation Acc: 55.94%
iteration 5000
   Validation Acc: 52.81%


In [12]:
# 모델 저장
saver = tf.train.Saver()
saver.save(sess, './MLmodels/ANN_model_tf1')

'./MLmodels/ANN_model_tf1'

## NN model 불러와서 진단

텐서플로우 그래프 초기화 (아무것도 작성안했을 시 실행 X)

In [13]:
# Tensorflow Graph 지울 때 사용
tf.reset_default_graph()

In [14]:
# input place holders
X = tf.placeholder(tf.float32, [None, Training_Fold1.shape[1]])
Y = tf.placeholder(tf.float32, [None, 2])

# dropout (keep_prob) rate 0.7 on training, but should be 1 for testing
keep_prob = tf.placeholder(tf.float32)

# weights & bias for nn layers

# Input Layer
W1 = tf.Variable(tf.random_normal([Training_Fold1.shape[1], noOfNeuron]))
b1 = tf.Variable(tf.random_normal([noOfNeuron]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

# Hidden Layer 1
W2 = tf.Variable(tf.random_normal([noOfNeuron, noOfNeuron]))
b2 = tf.Variable(tf.random_normal([noOfNeuron]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

# Hidden Layer 2
W3 = tf.Variable(tf.random_normal([noOfNeuron, noOfNeuron]))
b3 = tf.Variable(tf.random_normal([noOfNeuron]))
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)

# Output Layer
W4 = tf.Variable(tf.random_normal([noOfNeuron, 2]))
b4 = tf.Variable(tf.random_normal([2]))
hypothesis = tf.matmul(L3, W4) + b4


# define cost/loss &optimizer
cost      = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learningRate).minimize(cost)

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy           = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [15]:
# 모델 불러오기 (그래프에 Variable w, b 불러옴)
sess  = tf.Session()
LoadedModel = tf.train.Saver()
LoadedModel.restore(sess, './MLmodels/ANN_model_tf1')

INFO:tensorflow:Restoring parameters from ./MLmodels/ANN_model_tf1


In [16]:
Test_Data         = np.array(pd.read_csv('./K_FoldData/Test_Data' , sep=",", header=None))
Test_Label_forANN = np.array(pd.read_csv('./K_FoldData/Test_Label_forANN', sep=",", header=None))

In [17]:
Accuracy = sess.run(accuracy, feed_dict={X : Test_Data, Y: Test_Label_forANN})
print('[Performance of ANN model] \n')
print('Accuracy : {:.2f}%'.format(Accuracy*100))

[Performance of ANN model] 

Accuracy : 52.50%


In [18]:
Predict = sess.run(hypothesis, feed_dict={X : Test_Data, Y: Test_Label_forANN})
pd.DataFrame(Predict)

,0,1
0,6623.878418,6621.854004
1,8677.680664,8680.493164
2,11168.798828,11189.466797
3,32337.072266,32347.113281
4,7351.035645,7362.979492
5,37459.824219,37534.406250
6,12550.371094,12565.639648
7,7151.445801,7167.458984
8,13192.564453,13226.728516
9,17803.333984,17803.966797
